# Summary
- there are no benefits in using numba
- use `lagmat_2` as it supports row-oriented time series

# Load packages

In [1]:
import numpy as np
import numba
import csv

# Load Data

In [2]:
x1 = list(csv.reader(open('data1.csv', 'r')))
x1 = np.array(x1, dtype=float)

x2 = list(csv.reader(open('data2.csv', 'r')))
x2 = np.array(x2, dtype=float)

# lagmat prototypes

In [3]:
def lagmat_1(A: np.array, lags: list, order: str = 'F') -> np.array:
    # detect negative lags
    if min(lags) < 0:
        raise Exception((
            "Negative lags are not allowed. Only provided integers "
            "greater equal 0 as list/tuple elements"))

    # correct dimensions
    if len(A.shape) == 1:
        A = A.reshape(-1, 1)

    # number of colums and lags
    n_rows, n_cols = A.shape
    n_lags = len(lags)

    # allocate memory
    B = np.empty(
        shape=(n_rows, n_cols * n_lags),
        order=order,
        dtype=A.dtype)
    B[:] = np.nan

    # Copy lagged columns of A into B
    for i, l in enumerate(lags):
        # target columns of B
        j = i * n_cols
        k = j + n_cols  # (i+1) * ncols
        # number rows of A
        nl = n_rows - l
        # Copy
        B[l:, j:k] = A[:nl, :]

    return B

In [4]:
def lagmat_2(A: np.array, lags: list, orient: str = 'col') -> np.array:
    # detect negative lags
    if min(lags) < 0:
        raise Exception((
            "Negative lags are not allowed. Only provided integers "
            "greater equal 0 as list/tuple elements"))
    
    if orient in ('row', 'rows'):
        if len(A.shape) == 1:
            A = A.reshape(1, -1)
        A = np.array(A, order='C')
        return lagmat_rows_2(A, lags)
    
    elif orient in ('col', 'columns'):
        if len(A.shape) == 1:
            A = A.reshape(-1, 1)
        A = np.array(A, order='F')
        return lagmat_cols_2(A, lags)
    else:
        return None


def lagmat_rows_2(A: np.array, lags: list):
    # number of colums and lags
    n_rows, n_cols = A.shape
    n_lags = len(lags)
    # allocate memory
    B = np.empty(shape=(n_rows * n_lags, n_cols), order='C', dtype=A.dtype)
    B[:] = np.nan
    # Copy lagged columns of A into B
    for i, l in enumerate(lags):
        # target rows of B
        j = i * n_rows
        k = j + n_rows  # (i+1) * n_rows
        # number cols of A
        nc = n_cols - l
        # Copy
        B[j:k, l:] = A[:, :nc]
    return B


def lagmat_cols_2(A: np.array, lags: list):
    # number of colums and lags
    n_rows, n_cols = A.shape
    n_lags = len(lags)
    # allocate memory
    B = np.empty(shape=(n_rows, n_cols * n_lags), order='F', dtype=A.dtype)
    B[:] = np.nan
    # Copy lagged columns of A into B
    for i, l in enumerate(lags):
        # target columns of B
        j = i * n_cols
        k = j + n_cols  # (i+1) * ncols
        # number rows of A
        nl = n_rows - l
        # Copy
        B[l:, j:k] = A[:nl, :]
    return B

In [5]:
def lagmat_3(A: np.array, lags: np.array, orient: str = 'col') -> np.array:
    # detect negative lags
    if np.min(lags) < 0:
        raise Exception((
            "Negative lags are not allowed. Only provided integers "
            "greater equal 0 as list/tuple elements"))
    # enforce numpy
    #lags = np.array(lags)
    # enforce float dtype
    
    if orient in ('row', 'rows', 'C'):
        if len(A.shape) == 1:
            A = A.reshape(1, -1)
        #A = np.array(A, order='C')
        return lagmat_rows_3(A, lags)
    
    elif orient in ('col', 'columns', 'F'):
        if len(A.shape) == 1:
            A = A.reshape(1, -1)
        else:
            A = A.T            
        #A = np.array(A, order='C')
        return lagmat_rows_3(A, lags).T
    else:
        return None


@numba.jit(nopython=True, parallel=True)
def lagmat_rows_3(A: np.array, lags: np.array):
    # number of colums and lags
    n_rows, n_cols = A.shape
    n_lags = lags.shape[0]
    # allocate memory
    B = np.empty(shape=(n_rows * n_lags, n_cols), dtype=A.dtype)
    B[:] = np.nan
    # Copy lagged columns of A into B
    for i, l in enumerate(lags):
        # target rows of B
        j = i * n_rows
        k = j + n_rows  # (i+1) * n_rows
        # number cols of A
        nc = n_cols - l
        # Copy
        B[j:k, l:] = A[:, :nc]
    return B


# Pre-Run

In [6]:
lags = range(100)
lags2 = np.array(lags)
x1T = x1.T

print("column-oriented time series:")
%time _ = lagmat_1(x1, lags, order='F')
%time _ = lagmat_1(x1, lags, order='C')
%time _ = lagmat_2(x1, lags, orient='col')
%time _ = lagmat_3(x1, lags2, orient='col')

print("row-oriented time series:")
%time _ = lagmat_2(x1T, lags, orient='row')
%time _ = lagmat_3(x1T, lags2, orient='row')

column-oriented time series:
CPU times: user 374 ms, sys: 249 ms, total: 622 ms
Wall time: 627 ms
CPU times: user 1.39 s, sys: 327 ms, total: 1.71 s
Wall time: 1.96 s
CPU times: user 369 ms, sys: 315 ms, total: 684 ms
Wall time: 684 ms
CPU times: user 1.8 s, sys: 358 ms, total: 2.15 s
Wall time: 1.96 s
row-oriented time series:
CPU times: user 378 ms, sys: 324 ms, total: 701 ms
Wall time: 705 ms
CPU times: user 555 ms, sys: 320 ms, total: 875 ms
Wall time: 653 ms


# Speed Test

In [7]:
lags = range(100)
lags2 = np.array(lags)
x1T = x1.T

print("column-oriented time series:")
%timeit _ = lagmat_1(x1, lags, order='F')
%timeit _ = lagmat_1(x1, lags, order='C')
%timeit _ = lagmat_2(x1, lags, orient='col')
%timeit _ = lagmat_3(x1, lags2, orient='col')

print("\nrow-oriented time series:")
%timeit _ = lagmat_2(x1T, lags, orient='row')
%timeit _ = lagmat_3(x1T, lags2, orient='row')

column-oriented time series:
613 ms ± 6.88 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
1.69 s ± 210 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
612 ms ± 14 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
601 ms ± 19.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

row-oriented time series:
695 ms ± 68.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
602 ms ± 42.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [8]:
lags = range(100)
lags2 = np.array(lags)
x2T = x2.T

print("column-oriented time series:")
%timeit _ = lagmat_1(x2, lags, order='F')
%timeit _ = lagmat_1(x2, lags, order='C')
%timeit _ = lagmat_2(x2, lags, orient='col')
%timeit _ = lagmat_3(x2, lags2, orient='col')

print("\nrow-oriented time series:")
%timeit _ = lagmat_2(x2T, lags, orient='row')
%timeit _ = lagmat_3(x2T, lags2, orient='row')

column-oriented time series:
592 ms ± 5.56 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
548 ms ± 27.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
576 ms ± 5.05 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
546 ms ± 4.09 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

row-oriented time series:
573 ms ± 3.47 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
567 ms ± 47.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
